# Exchange Rate Crises (XRC)
Dario Caldara, Anna Lipinska, Henry Young, David Yu

## Introduction
Using data on bilateral exchange rates with the U.S. dollar from 1940 to 2020, we investigate the causes of currency crises in developing economies. This document displays various high-level overviews of currency crises. There are three types of graphics: bar charts of crisis incidence, maps showing total incidence over our time window, and maps showing dynamic incidence over time.

## Links
## Data

In [ ]:
# set up graphics production

# imports
import pandas as pd
import numpy as np
from plotly import express as px
from plotly.io import write_html
import plotly.graph_objects as go

# functions
def customLegend(fig, nameSwap):
    """
    Replace default legend entries with custom ones.
    Credit: https://stackoverflow.com/questions/64371174/plotly-how-to-change-variable-label-names-for-the-legend-in-a-plotly-express-li
    
    Parameters:
        fig : Plotly figure
        nameSwap : Python dict containing all current legend entries as keys and desired legend entries as values.
    
    Returns:
        modified fig object
    """
    for i, dat in enumerate(fig.data):
        for elem in dat:
            if elem == 'name' and fig.data[i].name in nameSwap:
                fig.data[i].name = nameSwap[fig.data[i].name]
    return fig

def time_series_indicators(df, indicators, **kwargs):
    """
    Plots the given indicators over the maximum possible timeframe.
    
    Parameters:
        df: pandas DataFrame in panel format, by country and year
        indicators: list of variable names to plot
        **kwargs: optional arguments to pass to plotly
        
    Returns:
        A plotly graph object, a bar chart showing incidence of
        each indicator over time.
    """
    
    # aggregate count of crises by year
    updated_indicators = []
    for i in indicators:
        temp = df.groupby("year")[i].agg('sum')
        df = pd.merge(df, temp, how = "left", on = "year")
        updated_indicators.append(i + "_y")
    
    fig = px.bar(df,
                 x = "year",
                 range_x = [1950, 2020],
                 y = updated_indicators,
                 barmode = 'overlay',
                 labels = {"variable" : "Threshold",
                           "value" : "Number of Crises",
                           "year" : "Year",
                           "x_ss": "Country Sample Size"},
                 hover_data = {"variable": False, "year": True, "x_ss": True},
                 title = "Number of Currency Crises, by Year <br><sup>Click on legend items to show/hide different thresholds</sup>")
    
    fig = customLegend(fig = fig, nameSwap = {"cc15_t_y" : "15%",
                                              "cc25_t_y" : "25%",
                                              "cc30_t_y" : "30%",
                                              "cc15_w_y" : "15% (3-year window)",
                                              "cc25_w_y" : "25% (3-year window)",
                                              "cc30_w_y" : "30% (3-year window)",
                                              "cc15_a_y" : "15% (3-year window, 10% acceleration)",
                                              "cc25_a_y" : "25% (3-year window, 10% acceleration)",
                                              "cc30_a_y" : "30% (3-year window, 10% acceleration)"})
    
    fig.update_layout(xaxis_title = None)
    
    # hide stricter thresholds initially
    to_hide = ["30%", "25%", "30% (3-year window)", "25% (3-year window)",
               "30% (3-year window, 10% acceleration)", "25% (3-year window, 10% acceleration)"]
    for i in to_hide:
        fig.update_traces(visible = "legendonly",
                          selector = dict(name = i))
    
    return fig

def static_indicator(df, indi, **kwargs):
    """
    Plots a specific indicator on a static map, showing total incidence.
    
    Parameters:
        df: pandas DataFrame in panel format, by country and year
        indi: str of one variable to plot
        **kwargs: optional arguments to pass to plotly
    
    Returns:
        plotly graph object plotting total incidence.
    """
    
    df_collapsed = df.groupby(["country", "iso_alpha"], as_index = False)[indi].aggregate("sum", min_count = 1)
    df_collapsed[indi + "_str"] = df_collapsed[indi].astype(str)
    
    # recode NaN values
    df_collapsed.replace(to_replace = "nan", value = "No Data", inplace = True)

    values = df_collapsed[indi + "_str"].unique()
    color_map = {}
    for v in values:
        assign = ""
        if v == "0.0": assign = "Blue"
        elif v == "1.0" or v == "2.0": assign = "Green"
        elif v == "3.0" or v == "4.0": assign = "Yellow"
        elif v == "5.0" or v == "6.0": assign = "Orange"
        elif v == "No Data": assign = "Gray"
        else: assign = "Crimson"
        color_map[v] = assign
    
    title_1 = indi[2:4]
    title_2 = indi[5] # character to indicate threshold, window, or acceleration
    if title_2 == "t": title_2 = ""
    elif title_2 == "w": title_2 = ", 3-year window"
    elif title_2 == "a": title_2 = ", 3-year window, 10% acceleration"
    else: pass
    
    fig = px.choropleth(df_collapsed,
                        locations = 'iso_alpha',
                        title = "Frequency of Currency Crises (" + title_1 + "% threshold" + title_2 + ") from 1940-2020",
                        color = indi + "_str",
                        color_discrete_map = color_map,
                        category_orders = {indi + "_str": ["7.0", "6.0", "5.0", "4.0", "3.0", "2.0", "1.0", "0.0", "No Data"]},
                        hover_name = "country",
                        hover_data = {"country": False, "iso_alpha": False},
                        labels = {"country": "Country", indi + "_str": "Number of Crisis-Years"})
    
    # construct dict for custom legend entries
    legend_map = {}
    for v in values:
        assign = ""
        if v == "0.0":
            assign = "0"
        elif v == "1.0":
            assign = "1-2"
        elif v == "3.0":
            assign = "3-4"
        elif v == "5.0":
            assign = "5-6"
        elif v == "7.0":
            assign = "7+"
        elif v == "No Data":
            assign = "No Data"
        else:
            assign = "hide"
        legend_map[v] = assign
    
    customLegend(fig, legend_map)

    fig.update_traces(showlegend = False, selector = dict(name = "hide"))
    
    return fig

def dynamic_indicator(df, indi, **kwargs):
    """
    Plots a specific indicator on a dynamic map with a time slider.
    
    Parameters:
        df: pandas DataFrame in panel format, by country and year
        indi: str of one variable to plot
        **kwargs: optional arguments to pass to plotly
        
    Returns:
        plotly graph object plotting incidence over time.
    """
    
    df["show"] = df[indi]
    df["show"] = df["show"].replace(np.nan, "No Data")
    df["show"] = df["show"].replace(0, "No Crisis")
    df["show"] = df["show"].replace(1, "Crisis")

    # add reference rows at the top (not for display!)
    df.loc[-2] = ['null0', 1940, "00", "000", np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, "No Crisis"]  # adding a row
    df.loc[-1] = ['null1', 1940, "01", "001", np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, "Crisis"]  # adding a row
    df.index = df.index + 2  # shifting index
    df.sort_index(inplace=True)

    title_1 = indi[2:4]
    title_2 = indi[5] # character to indicate threshold, window, or acceleration
    if title_2 == "t": title_2 = ""
    elif title_2 == "w": title_2 = ", 3-year window"
    elif title_2 == "a": title_2 = ", 3-year window, 10% acceleration"
    else: pass
    
    fig = px.choropleth(df,
                        locations = 'iso_alpha',
                        color = "show",
                        color_discrete_map = {
                            "No Data": "Gray",
                            "No Crisis": "lightskyblue",
                            "Crisis": "Red"
                        },
                        title = "Dynamic Incidence of Exchange Rate Crises (" + title_1 + "% threshold" + title_2 + ")",
                        animation_frame = "year",
                        animation_group = "country",
                        category_orders = {"show": ["No Data", "No Crisis", "Crisis"]},
                        hover_data = {"show": False, "iso_alpha": False, "year": False, "country": True})

    fig.update_layout(legend_title_text = "Threshold:" + indi[2:4] + "%")
    
    return fig

# read in data
data = pd.read_csv("xrc.csv")
data.rename(columns = {"iso3": "iso_alpha"}, inplace = True)

## Bar Charts

In [ ]:
df = data.copy()
fig = time_series_indicators(df, ["cc15_t", "cc25_t", "cc30_t"])
fig.show()

In [ ]:
df = data.copy()
fig = time_series_indicators(df, ["cc15_a", "cc25_a", "cc30_a"])
fig.show()

## Static Incidence

In [ ]:
fig = static_indicator(df, "cc15_t")
fig.show()

In [ ]:
fig = static_indicator(df, "cc25_t")
fig.show()

In [ ]:
fig = static_indicator(df, "cc30_t")
fig.show()

In [ ]:
fig = static_indicator(df, "cc15_a")
fig.show()

In [ ]:
fig = static_indicator(df, "cc25_a")
fig.show()

In [ ]:
fig = static_indicator(df, "cc30_a")
fig.show()

## Dynamic Incidence

In [ ]:
fig = dynamic_indicator(df, "cc15_t")
fig.show()

In [ ]:
fig = dynamic_indicator(df, "cc25_t")
fig.show()

In [ ]:
fig = dynamic_indicator(df, "cc30_t")
fig.show()

In [ ]:
fig = dynamic_indicator(df, "cc15_a")
fig.show()

In [ ]:
fig = dynamic_indicator(df, "cc25_a")
fig.show()

In [ ]:
fig = dynamic_indicator(df, "cc30_a")
fig.show()

In [ ]:
!jupyter nbconvert --to html --TemplateExporter.exclude_input=True currency-crises.ipynb